In [1]:
import pandas as pd
import numpy as np
import folium
import json
import re

## Filter Function
def mask(df, key, value):
    return df[df[key] == value]
pd.DataFrame.mask = mask

In [2]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.executor.memory", "16g"))
sc = SparkContext(conf = conf)
#sc = pyspark.SparkContext()
from pyspark.sql.functions import udf
from pyspark.sql import SQLContext
from pyspark.sql.functions import mean, min, max
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, FloatType,ArrayType,DataType




In [3]:
sc.version

u'1.6.2'

In [3]:
#####################################################
###########Input Tone, Location Raw Data#############

sqlCtx = SQLContext(sc)
#df = sqlCtx.read.format('com.databricks.spark.csv').option("header", "true").load("Data/General/*.csv") 
#df = sqlCtx.read.format('com.databricks.spark.csv').option("header", "true").load("Data/General/domain_location0.csv")
df = sqlCtx.read.format('com.databricks.spark.csv').option("header", "true").load("Data/Theme/spark/*.csv") 

#df = sqlCtx.read.load('Data/domain_location0.csv', )
#df = sqlCtx.read.load(source="com.databricks.spark.csv", header="true", path = "Data/domain_location0.csv")
#df = sc.textFile("Data/domain_location0.csv")
#df = sqlCtx.load(source="csv", path = "Data/domain_location0.csv")
sourceCountry = sqlCtx.read.format('com.databricks.spark.csv')\
                .option("header", "true").load('Data/sourceCountry.csv')


In [5]:
df = df.dropna()

In [4]:
tone = df.join(sourceCountry, df.SourceCommonName == sourceCountry.Domain, 'left')

In [5]:
tone.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- CountryName: string (nullable = true)



In [5]:
#tone.select([min('DATE'), max('DATE')]).show()

In [7]:
tone = tone.dropna()

In [12]:
tone.count()

24255462

In [6]:
#china = tone[tone['CountryName'] == 'China']
us = tone[tone['CountryName'] == 'United States']
#uk = tone[tone['CountryName'] == 'United Kingdom']

In [14]:
us.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- CountryName: string (nullable = true)
 |-- Tone: float (nullable = true)



In [22]:
us.count()

253799

In [ ]:
china.count()

In [ ]:
uk.count()

In [7]:

######################################################
###############Clean the Tone Data####################
#from pyspark.sql.functions import UserDefinedFunction
#pattern = re.compile("^\s+|\s*,\s*|\s+$")

#udf = UserDefinedFunction(lambda x: float(pattern.split(x)[0]))

pattern = re.compile("^\s+|\s*,\s*|\s+$")
#def cleanTone(Tone):
    #return 
    
UDF = udf(lambda x: float(pattern.split(x)[0]),FloatType())

#china = china.withColumn('Tone',UDF(china.V2Tone))
us = us.withColumn('Tone',UDF(us.V2Tone))
#uk = uk.withColumn('Tone',UDF(uk.V2Tone))

#us = us.map(lambda x:float(pattern.split(x.ToneV2)[0]))

In [ ]:

us.select([mean('Tone'), min('Tone'), max('Tone')]).show()

In [ ]:
china.select([mean('Tone'), min('Tone'), max('Tone')]).show()

In [ ]:
uk.select([mean('Tone'), min('Tone'), max('Tone')]).show()

In [8]:
###########################################################
#######Clean the Location data mentioned in the news#######

def cleanTargetCountry(Loc,Tone):
    count={}
    result = {}
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        if  key not in count.keys():
            count[key] = 1
        else:
            count[key] += 1
    totalCount = float(sum(count.values()))
    for i in Loc.split(';'):
        key = i.split('#')[2][:2]
        result[key] = count[key] / totalCount * Tone
        
    result = json.dumps(result)
    return result

#UDF2 = udf(lambda x,y:cleanTargetCountry(x,y))

UDF2 = udf(cleanTargetCountry)
us = us.withColumn('Target',UDF2(us.Locations,us.Tone))
#china = china.withColumn('Target',UDF2(china.Locations,china.Tone))
#uk = uk.withColumn('Target',UDF2(uk.Locations,uk.Tone))


#china['Location1'] = china.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
#us['Location1'] = us.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)
#uk['Location1'] = uk.apply(lambda x: cleanTargetCountry(x['Location'],x['Tone']), axis = 1)

In [9]:
us.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- CountryName: string (nullable = true)
 |-- Tone: float (nullable = true)
 |-- Target: string (nullable = true)



In [10]:
from pyspark.sql.functions import array_contains

theme = 'SCANDAL'

pattern = re.compile("^\s+|\s*,\s*|\s+$")
#tone['Tone'] = tone['Tone'].apply(lambda x:float(pattern.split(x)[0]))
#test1 = test.ix[2,'Theme'].split(';')
#pattern.split(test1[1])[0]

def themeFilter(row):
    themeList = row.split(';')
    #print themeList
    for i in range(len(themeList)):
        themeList[i] = pattern.split(themeList[i])[0]
    return themeList
UDF3 = udf(themeFilter,ArrayType(StringType()))
us = us.withColumn('Themes',UDF3(us.V2Themes))
#us = us.withColumn('Themes1',json.loads(us.Themes))
#us_Filter = us.where(col("V2Themes").likes("SCANDAL"))

In [24]:
us.select('Themes').show(2)

+--------------------+
|              Themes|
+--------------------+
|[MEDIA_SOCIAL, TA...|
|[IDEOLOGY, TAX_FN...|
+--------------------+
only showing top 2 rows



In [11]:
us_themeFilter = us.where(array_contains(us.Themes, "SCANDAL"))

In [12]:
us_themeFilter.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- SourceCommonName: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- V2Tone: string (nullable = true)
 |-- V2Themes: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- CountryName: string (nullable = true)
 |-- Tone: float (nullable = true)
 |-- Target: string (nullable = true)
 |-- Themes: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [40]:
us_themeFilter.count()

78314

In [15]:
#usPandas = us_themeFilter.toPandas()

In [13]:
###########################################################
##################Initial Result Dataframe#################

## China
countryInfo = pd.read_csv('Data/countryInfo.csv', index_col = False, low_memory = False)

china_result = countryInfo[['fips','country']]
china_result['tone'] = 0
china_result['nb_article'] = 0
china_result = china_result.set_index('fips')

## U.S.
us_result = countryInfo[['fips','country']]
us_result['tone'] = 0
us_result['nb_article'] = 0
us_result = us_result.set_index('fips')

## U.K
uk_result = countryInfo[['fips','country']]
uk_result['tone'] = 0
uk_result['nb_article'] = 0
uk_result = uk_result.set_index('fips')




/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dawei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/i

In [11]:
us = us.cache()


In [16]:
china = china.cache()
uk = uk.cache()

In [17]:
def f(iterator):
    for x in iterator:
        print str(x)
        
#test = sc.parallelize([1, 2, 3, 4, 5])
#test.foreachPartition(f)

In [12]:
us.cache().rdd

MapPartitionsRDD[37] at javaToPython at NativeMethodAccessorImpl.java:-2

In [13]:
us.rdd.unpersist()

MapPartitionsRDD[37] at javaToPython at NativeMethodAccessorImpl.java:-2

In [14]:
tone.rdd.unpersist()

MapPartitionsRDD[59] at javaToPython at NativeMethodAccessorImpl.java:-2

In [25]:
###########################################################
###########Calculate Number of Articles for each target####
#############Maintain the tone result to result df#########

for row in us_themeFilter.rdd.collect():
    target = json.loads(row['Target'])
    for country in target.keys():
        try:
            us_result.ix[country,'tone'] += target[country]
            us_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass

'''   
for row in uk.rdd.collect():
    target = json.loads(row['Target'])
    for country in target.keys():
        try:
            uk_result.ix[country,'tone'] += target[country]
            uk_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
for row in china.rdd.collect():
    target = json.loads(row['Target'])
    for country in target.keys():
        try:
            china_result.ix[country,'tone'] += target[country]
            china_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass

def usCollect(iterator):
    for x in iterator:
        target = json.loads(x)
        for country in target.keys():
            try:
                us_result.ix[country,'tone'] += target[country]
                us_result.ix[country,'nb_article'] +=1
            except KeyError:
                pass

def usCollect(Tone):
    
    target = json.loads(Tone)
    for country in target.keys():
        try:
            us_result.ix[country,'tone'] += target[country]
            us_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass

def chinaCollect(x):
    target = json.loads(x['Target'])
    for country in target.keys():
        try:
            china_result.ix[country,'tone'] += target[country]
            china_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
def ukCollect(x):
    target = json.loads(x['Target'])
    for country in target.keys():
        try:
            uk_result.ix[country,'tone'] += target[country]
            uk_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
'''        
#UDF3 = udf(usCollect)

#usRdd = us.rdd.repartition(2000)
#usRdd.getNumPartitions()      
#usRdd.foreachPartition(usCollect)
#UDF3(us.Tone)
    

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 4.0 failed 1 times, most recent failure: Lost task 2.0 in stage 4.0 (TID 1133, localhost): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at net.razorvine.pickle.PickleUtils.readbytes(PickleUtils.java:54)
	at net.razorvine.pickle.Unpickler.load_long_binput(Unpickler.java:655)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:236)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1$$anonfun$apply$3.apply(python.scala:412)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1$$anonfun$apply$3.apply(python.scala:411)
	at scala.collection.Iterator$$anon$13.hasNext(Iterator.scala:371)
	at scala.collection.Iterator$$anon$

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49862)
----------------------------------------


Traceback (most recent call last):
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/dawei/spark-1.6.2-bin-hadoop2.6/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/home/dawei/spark-1.6.2-bin-hadoop2.6/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


In [15]:
def usCollect(x):
    target = json.loads(x['Target'])
    for country in target.keys():
        try:
            us_result.ix[country,'tone'] += target[country]
            us_result.ix[country,'nb_article'] +=1
        except KeyError:
            pass
        
#us.rdd.map(usCollect).collect()
#UDF3 = udf(usCollect)
usRdd = us_themeFilter.rdd.repartition(2000)
usRdd.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
#usRdd.getNumPartitions()
usRdd.foreachPartition(usCollect)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 4.0 failed 1 times, most recent failure: Lost task 2.0 in stage 4.0 (TID 1133, localhost): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at scala.collection.mutable.ResizableArray$class.ensureSize(ResizableArray.scala:99)
	at scala.collection.mutable.ArrayBuffer.ensureSize(ArrayBuffer.scala:47)
	at scala.collection.mutable.ArrayBuffer.$plus$eq(ArrayBuffer.scala:83)
	at scala.collection.mutable.ArrayBuffer.$plus$eq(ArrayBuffer.scala:47)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:244)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:244)
	at scala.collection.Iterator$class.foreach(Iterator.scala:727)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1157)
	at scala.collection.IterableLike$class.foreach(IterableLike

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60086)
----------------------------------------


Traceback (most recent call last):
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/dawei/anaconda2/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/home/dawei/spark-1.6.2-bin-hadoop2.6/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/home/dawei/spark-1.6.2-bin-hadoop2.6/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError


In [36]:
us


DataFrame[DATE: string, SourceCommonName: string, Locations: string, V2Tone: string, V2Themes: string, Domain: string, FIPS: string, CountryName: string, Tone: float, Target: string]

In [17]:
######Calculate Average tone for other countries for China, US, UK


us_result['tone'] = us_result['tone']/us_result['nb_article']
#china_result['tone'] = china_result['tone']/china_result['nb_article']
#uk_result['tone'] = uk_result['tone']/uk_result['nb_article']
us_result.describe()

/home/dawei/.local/lib/python2.7/site-packages/numpy/lib/function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,tone,nb_article
count,0.0,245.0
mean,NaN,0.0
std,NaN,0.0
min,NaN,0.0
25%,NaN,0.0
50%,NaN,0.0
75%,NaN,0.0
max,NaN,0.0


In [47]:
us_result.head(20)

,country,tone,nb_article
fips,,,
AN,Andorra,0,0
AE,United Arab Emirates,0,0
AF,Afghanistan,0,0
AC,Antigua and Barbuda,0,0
AV,Anguilla,0,0
AL,Albania,0,0
AM,Armenia,0,0
AO,Angola,0,0
AY,Antarctica,0,0


In [ ]:
us_result.sort(columns = 'tone',ascending=False)[:20]

In [ ]:
###########################################################
##################Filter Data##############################

## China

china_result = china_result.dropna()
china_result = china_result.sort(columns = 'nb_article',ascending=False)[:200]
china_result = pd.merge(china_result,countryGeo, on = ['country'],how = 'left')
china_result = china_result.dropna()
china_result = china_result[china_result['nb_article']>1000]

## US
us_result = us_result.dropna()
us_result = us_result.sort(columns = 'nb_article',ascending=False)[:200]
us_result = pd.merge(us_result,countryGeo, on = ['country'],how = 'left')
us_result = us_result.dropna()
us_result = us_result[us_result['nb_article']>1000]

## UK
uk_result = uk_result.dropna()
uk_result = uk_result.sort(columns = 'nb_article',ascending=False)[:200]
uk_result = pd.merge(uk_result,countryGeo, on = ['country'],how = 'left')
uk_result = uk_result.dropna()
uk_result = uk_result[uk_result['nb_article']>1000]

In [ ]:
china_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
us_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)
uk_Map = folium.Map(location=[35.85, 104.19], zoom_start=1)

In [ ]:
import folium

'''
for index, row in test2.iterrows():
    print index
    Location = [row['lat'],row['lon']]
    name = row['country']
    R = abs(row.tone)
    if row.tone>0:
        Color = '#3186cc'
    else:
        Color = '#cc3131'
    if name == 'Turkey':
        print name
    if index>30:
        folium.CircleMarker(location=Location, radius=1000000*R,
                            popup=name,
                            color=Color,
                            fill_color=Color).add_to(Map)

'''
def draw(Map,loc,r,name):
    if r > 0.05:
        Color = '#3186cc'
    elif r < -0.05:
        Color = '#cc3131'
    else:
        Color = '#0ed148'#0ed148
    
    folium.CircleMarker(location=loc, radius=700000*abs(r),
                            popup=name+': '+str(r),
                            color=Color,
                            fill_color=Color).add_to(Map)
    
china_result.apply(lambda x: draw(china_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)
us_result.apply(lambda x: draw(us_Map,[x['lat_x'],x['lon_x']],x['tone'],x['country']),axis = 1)
uk_result.apply(lambda x: draw(uk_Map,[x['lat'],x['lon']],x['tone'],x['country']),axis = 1)

        

china_Map

In [ ]:
us_Map

In [ ]:
uk_Map

In [ ]:
china_Map.save('theme_china.html')
us_Map.save('theme_us.html')
uk_Map.save('theme_uk.html')